In [2]:
import pandas as pd
import sqlite3
import requests
import re
import nltk
import numpy as np
import string

In [3]:
# https://stackoverflow.com/questions/4636970/sqlite3-operationalerror-unable-to-open-database-file/4637055
db_path = 'C:\\users\\jordanhiatt\\ETAClass\\gutenberg.db'

In [4]:
con = sqlite3.connect('gutenberg.db')
con

In [5]:
# https://datacarpentry.org/python-ecology-lesson/09-working-with-sql/index.html
cur = con.cursor()

# are there any other spellings of Arthur Conan Doyle?  Looks like only one spelling/format
for row in cur.execute("SELECT DISTINCT creators from 'catalog' WHERE creators LIKE '%Doyle%' AND creators LIKE '%Arthur Conan%';"):
    print(row)

('DOYLE, ARTHUR CONAN',)


In [6]:
# get a unique list of titles and gid's
gid_titles = list(cur.execute("""
SELECT DISTINCT gid, creators, title 
FROM (SELECT * FROM 'catalog' GROUP BY title)
WHERE creators = 'DOYLE, ARTHUR CONAN' AND languages = "EN";
"""))
for row in gid_titles[:10]:
    print(row)

(21768, 'DOYLE, ARTHUR CONAN', 'A DESERT DRAMA: BEING THE TRAGEDY OF THE "KOROSKO"')
(5260, 'DOYLE, ARTHUR CONAN', 'A DUET, WITH AN OCCASIONAL CHORUS')
(244, 'DOYLE, ARTHUR CONAN', 'A STUDY IN SCARLET')
(9874, 'DOYLE, ARTHUR CONAN', 'A VISIT TO THREE FRONTS: JUNE 1916')
(48320, 'DOYLE, ARTHUR CONAN', 'ADVENTURES OF SHERLOCK HOLMES ILLUSTRATED')
(356, 'DOYLE, ARTHUR CONAN', 'BEYOND THE CITY')
(22357, 'DOYLE, ARTHUR CONAN', 'DANGER! AND OTHER STORIES')
(8634, 'DOYLE, ARTHUR CONAN', 'HIS LAST BOW')
(2350, 'DOYLE, ARTHUR CONAN', 'HIS LAST BOW: AN EPILOGUE OF SHERLOCK HOLMES')
(58574, 'DOYLE, ARTHUR CONAN', 'INDEX OF THE PROJECT GUTENBERG WORKS OF ARTHUR CONAN DOYLE')


In [7]:
# https://www.kite.com/python/answers/how-to-make-a-list-into-a-comma-separated-string-in-python
gid_file = []
for row in gid_titles:
    s = [str(element) for element in row]
    joined = "|".join(s)
    gid_file.append(joined)
gid_file[:10]

['21768|DOYLE, ARTHUR CONAN|A DESERT DRAMA: BEING THE TRAGEDY OF THE "KOROSKO"',
 '5260|DOYLE, ARTHUR CONAN|A DUET, WITH AN OCCASIONAL CHORUS',
 '244|DOYLE, ARTHUR CONAN|A STUDY IN SCARLET',
 '9874|DOYLE, ARTHUR CONAN|A VISIT TO THREE FRONTS: JUNE 1916',
 '48320|DOYLE, ARTHUR CONAN|ADVENTURES OF SHERLOCK HOLMES ILLUSTRATED',
 '356|DOYLE, ARTHUR CONAN|BEYOND THE CITY',
 '22357|DOYLE, ARTHUR CONAN|DANGER! AND OTHER STORIES',
 '8634|DOYLE, ARTHUR CONAN|HIS LAST BOW',
 '2350|DOYLE, ARTHUR CONAN|HIS LAST BOW: AN EPILOGUE OF SHERLOCK HOLMES',
 '58574|DOYLE, ARTHUR CONAN|INDEX OF THE PROJECT GUTENBERG WORKS OF ARTHUR CONAN DOYLE']

In [8]:
# https://github.com/ontoligent-design/pgtk

gut_utf8 = 'https://www.gutenberg.org/ebooks/{}.txt.utf-8'

def download_epubs(doc_list, outdir=None, sep='|'):
# def download_epubs(epub_file, outdir=None):
    """Download epubs from a list of files generated from a search"""
     
    doc_names = []

    if not outdir:
        outdir="books"       
    
    print("Downloading files to", outdir)
    for line in doc_list:
        row = line.split(sep)
        gid = row[0]
        try:
            int(gid)         
            url = gut_utf8.format(gid)
            r = requests.get(url)
            filename = '_'.join(row[1:3]).strip()
            filename = re.sub(r'\W+', '_', filename)
            filename = re.sub(r'_+', '_', filename)
            print(gid, filename)
            doc_name = "{}/{}-{}.txt".format(outdir, filename, gid)
            with open(doc_name, 'w') as outfile:
                outfile.write(r.text)
            doc_names.append((gid, filename, doc_name))
        except ValueError as e:
            print('#', gid, "not a GID")
    return doc_names

In [9]:
doc_names = download_epubs(gid_file)

21768 DOYLE_ARTHUR_CONAN_A_DESERT_DRAMA_BEING_THE_TRAGEDY_OF_THE_KOROSKO_
5260 DOYLE_ARTHUR_CONAN_A_DUET_WITH_AN_OCCASIONAL_CHORUS
244 DOYLE_ARTHUR_CONAN_A_STUDY_IN_SCARLET
9874 DOYLE_ARTHUR_CONAN_A_VISIT_TO_THREE_FRONTS_JUNE_1916
48320 DOYLE_ARTHUR_CONAN_ADVENTURES_OF_SHERLOCK_HOLMES_ILLUSTRATED
356 DOYLE_ARTHUR_CONAN_BEYOND_THE_CITY
22357 DOYLE_ARTHUR_CONAN_DANGER_AND_OTHER_STORIES
8634 DOYLE_ARTHUR_CONAN_HIS_LAST_BOW
2350 DOYLE_ARTHUR_CONAN_HIS_LAST_BOW_AN_EPILOGUE_OF_SHERLOCK_HOLMES
58574 DOYLE_ARTHUR_CONAN_INDEX_OF_THE_PROJECT_GUTENBERG_WORKS_OF_ARTHUR_CONAN_DOYLE
9504 DOYLE_ARTHUR_CONAN_MICAH_CLARKE_HIS_STATEMENT_AS_MADE_TO_HIS_THREE_GRANDCHILDREN_JOSEPH_GERVAS_AND_REUBEN_DURING_THE_HARD_WINTER_OF_1734


After removing duplicate titles and foreign languages, I've downloaded about 69 of the 130 or more Arthur Conan Doyle's works listed on Project Gutenberg.  However, some of these returned error pages and many others are short stories that might be duplicated in diffferent documents.  The next step will be to pull in this data in OCHO format. While we do this we will eliminate documents that are HTML formatted (the error pages) and will look for tables of contents that match.

In [ ]:
# I've decided to skip chapters for now because of inconsistent formatting.  I'll see if I need it.
OHCO = ['book_id', 'para_num', 'sent_num', 'token_num']

In [ ]:
# remove metadata
def stripMetadata(df):

    # general clean up
    df[OHCO[1]] = df[OHCO[1]].str.replace('—', ' — ')
    df[OHCO[1]] = df[OHCO[1]].str.replace('-', ' - ')

    # find begining line
    a = df[OHCO[1]].str.match(r"\*\*\*\s*START OF (THE|THIS) PROJECT")
    a_1 = df[OHCO[1]].str.match(r"\*END THE SMALL PRINT") #backup start for when the above isn't used

    if (df.loc[a].empty):
        an = df.loc[a_1].index[0]
    else:
        an = df.loc[a].index[0]

    # find ending line
    b = df[OHCO[1]].str.match(r"\*\*\*\s*END OF (THE|THIS) PROJECT")
    if (df.loc[b].empty):
        bn = df.tail(1).index[0]
    else:
        bn = df.loc[b].index[0]
        # print(df.loc[b])

    return(df.loc[an + 1 : bn - 2])

In [ ]:
def splitParagraphs(df, id):
    # dataframes have columns OHCO[0] and OHCO[1]
    # paragraph will be OHCO[-3]
    # id should be book_id


    df_p = df[OHCO[1]].str.split(r'\n\n+', expand=True).stack().to_frame().rename(columns={0:'para_str'})
    df_p['para_str'] = df_p['para_str'].str.replace(r'\n', ' ').str.strip()
    df_p = df_p[~df_p['para_str'].str.match(r'^\s*$')] # Remove empty paragraphs

    df_p = df_p.reset_index(drop=True)
    df_p.index.names = [OHCO[1]]
    df_p = df_p.reset_index()
    df_p[OHCO[0]]=id
    df_p.set_index(OHCO[:2], inplace=True)

    return df_p

In [ ]:
def cleanString(row):
    # remove numbers and punctuation
    # https://www.codegrepper.com/code-examples/python/pandas+series+remove+punctuation
    
    s = string.punctuation + '0123456789' + "″"
    for x in s:
        row = row.replace(x,'')
    return row


In [ ]:
def getLibSeries(row,df):
    #format the LIBRARY table
    # customized for Arthur Conan Doyle books currently
    
    book_id = row[0]
    book_title = row[1].replace('_',' ').strip()
    book_file = row[2]
    author = "Doyle, Arthur Conan"
    title = row[1][19:].replace('_',' ').strip()
    libSeries = pd.Series([book_id, book_title, book_file, author, title], index=df.columns)

    return libSeries

In [ ]:
df = pd.DataFrame(columns=['para_str', OHCO[0], OHCO[1]])
df = df.set_index(OHCO[:2])
LIBRARY = pd.DataFrame(columns=['book_id','book_title','book_file','author','title'])
for doc in doc_names:
    # pull in data
    epub = open(doc[2], 'r', encoding='utf-8-sig').readlines()
    # print(epub)

    # remove error pages
    if not (epub[0].startswith('<!DOCTYPE html>')):
        # strip metadata
        book = pd.DataFrame({OHCO[0]: doc[0], OHCO[1]: epub})

        book = stripMetadata(book)

        book = splitParagraphs(book, doc[0])

        book['para_str'] = book['para_str'].map(cleanString)

        libSeries = getLibSeries(doc, LIBRARY)
        
        LIBRARY = LIBRARY.append(libSeries, ignore_index=True) #https://www.kite.com/python/answers/how-to-append-a-list-as-a-row-to-a-pandas-dataframe-in-python

        # save to dataframe
        df = df.append(book)
    else:
        print("GID: {}, Title: {} was not found in Project Gutenberg".format(doc[0], doc[1]))

GID: 8634, Title: DOYLE_ARTHUR_CONAN_HIS_LAST_BOW was not found in Project Gutenberg
GID: 58574, Title: DOYLE_ARTHUR_CONAN_INDEX_OF_THE_PROJECT_GUTENBERG_WORKS_OF_ARTHUR_CONAN_DOYLE was not found in Project Gutenberg
GID: 54109, Title: DOYLE_ARTHUR_CONAN_ROUND_THE_FIRE_STORIES was not found in Project Gutenberg
GID: 8619, Title: DOYLE_ARTHUR_CONAN_ROUND_THE_RED_LAMP was not found in Project Gutenberg
GID: 59809, Title: DOYLE_ARTHUR_CONAN_THE_CASE_FOR_SPIRIT_PHOTOGRAPHY_WITH_CORROBORATIVE_EVIDENCE_BY_EXPERIENCED_RESEARCHERS_AND_PHOTOGRAPHERS was not found in Project Gutenberg
GID: 26153, Title: DOYLE_ARTHUR_CONAN_THE_LAST_OF_THE_LEGIONS_AND_OTHER_TALES_OF_LONG_AGO was not found in Project Gutenberg
GID: 834, Title: DOYLE_ARTHUR_CONAN_THE_MEMOIRS_OF_SHERLOCK_HOLMES was not found in Project Gutenberg
GID: 8618, Title: DOYLE_ARTHUR_CONAN_THE_PARASITE was not found in Project Gutenberg


In [ ]:
LIBRARY = LIBRARY.set_index('book_id')
LIBRARY

,book_title,book_file,author,title
book_id,,,,
21768,DOYLE ARTHUR CONAN A DESERT DRAMA BEING THE TR...,books/DOYLE_ARTHUR_CONAN_A_DESERT_DRAMA_BEING_...,"Doyle, Arthur Conan",A DESERT DRAMA BEING THE TRAGEDY OF THE KOROSKO
5260,DOYLE ARTHUR CONAN A DUET WITH AN OCCASIONAL C...,books/DOYLE_ARTHUR_CONAN_A_DUET_WITH_AN_OCCASI...,"Doyle, Arthur Conan",A DUET WITH AN OCCASIONAL CHORUS
244,DOYLE ARTHUR CONAN A STUDY IN SCARLET,books/DOYLE_ARTHUR_CONAN_A_STUDY_IN_SCARLET-24...,"Doyle, Arthur Conan",A STUDY IN SCARLET
9874,DOYLE ARTHUR CONAN A VISIT TO THREE FRONTS JUN...,books/DOYLE_ARTHUR_CONAN_A_VISIT_TO_THREE_FRON...,"Doyle, Arthur Conan",A VISIT TO THREE FRONTS JUNE 1916
48320,DOYLE ARTHUR CONAN ADVENTURES OF SHERLOCK HOLM...,books/DOYLE_ARTHUR_CONAN_ADVENTURES_OF_SHERLOC...,"Doyle, Arthur Conan",ADVENTURES OF SHERLOCK HOLMES ILLUSTRATED
...,...,...,...,...
39718,DOYLE ARTHUR CONAN THE WANDERINGS OF A SPIRITU...,books/DOYLE_ARTHUR_CONAN_THE_WANDERINGS_OF_A_S...,"Doyle, Arthur Conan",THE WANDERINGS OF A SPIRITUALIST
24951,DOYLE ARTHUR CONAN THE WAR IN SOUTH AFRICA ITS...,books/DOYLE_ARTHUR_CONAN_THE_WAR_IN_SOUTH_AFRI...,"Doyle, Arthur Conan",THE WAR IN SOUTH AFRICA ITS CAUSE AND CONDUCT
903,DOYLE ARTHUR CONAN THE WHITE COMPANY,books/DOYLE_ARTHUR_CONAN_THE_WHITE_COMPANY-903...,"Doyle, Arthur Conan",THE WHITE COMPANY


In [ ]:
df.shape

(305926, 1)

In [ ]:
df.head()

para_str
book_id para_num                            
21768   0           Produced by David Widger
        1                     A DESERT DRAMA
        2                              BEING
        3         The Tragedy of the Korosko
        4                                 BY

In [ ]:
# confiming we have multiple books here
df.sample(5)

,,para_str
book_id,para_num,
7964,3349,we have had occasion to talk of more than once...
9504,1158,Hows that eh Waller himself could not have cap...
13152,7018,Do light a cigarette said Mrs Scully
24951,3103,Bethulie and Edenburg in the Orange Free State
5260,1793,the only visitors at the Swiss hotel


In [ ]:
# remove punctuation
# import string
# df['para_str'] = df['para_str'].map(lambda x: x.translate(None, string.punctuation)) #https://stackoverflow.com/questions/15547409/how-to-get-rid-of-punctuation-using-nltk-tokenizer
string.punctuation
df[df.para_str == '9']

,,para_str
book_id,para_num,


In [ ]:
# Grab contents tables to see if there are any repeated short stories
# !!!!!!!!!!!!!!!!!! reevaluate this, I may not be able to do anything here with the time I have


In [ ]:
def tokenize(doc_df, OHCO=OHCO, remove_pos_tuple=False, ws=False):
    
    # Paragraphs to Sentences
    df = doc_df.para_str\
        .apply(lambda x: pd.Series(nltk.sent_tokenize(x)))\
        .stack()\
        .to_frame()\
        .rename(columns={0:'sent_str'})
    
    # Sentences to Tokens
    # Local function to pick tokenizer
    def word_tokenize(x):
        if ws:
            s = pd.Series(nltk.pos_tag(nltk.WhitespaceTokenizer().tokenize(x)))
        else:
            s = pd.Series(nltk.pos_tag(nltk.word_tokenize(x)))
        return s
            
    df = df.sent_str\
        .apply(word_tokenize)\
        .stack()\
        .to_frame()\
        .rename(columns={0:'pos_tuple'})
    
    # Grab info from tuple
    df['pos'] = df.pos_tuple.apply(lambda x: x[1])
    df['token_str'] = df.pos_tuple.apply(lambda x: x[0])
    if remove_pos_tuple:
        df = df.drop('pos_tuple', 1)
    
    # Add index
    df.index.names = OHCO
    
    return df

In [ ]:
TOKEN = tokenize(df,ws=True)

<ipython-input-372-690f95e99429>:5: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  .apply(lambda x: pd.Series(nltk.sent_tokenize(x)))\


In [ ]:
TOKEN.head(5)

pos_tuple  pos token_str
book_id para_num sent_num token_num                                
21768   0        0        0          (Produced, VBN)  VBN  Produced
                          1                 (by, IN)   IN        by
                          2             (David, NNP)  NNP     David
                          3            (Widger, NNP)  NNP    Widger
        1        0        0                  (A, DT)   DT         A

In [ ]:
TOKEN.sort_values('token_str')

pos_tuple  pos token_str
book_id para_num sent_num token_num                         
9874    35       0        0           (A, DT)   DT         A
34627   3537     0        7           (A, DT)   DT         A
        3522     0        3          (A, NNP)  NNP         A
48320   734      0        4          (A, NNP)  NNP         A
34627   3458     0        7          (A, NNP)  NNP         A
...                                       ...  ...       ...
48320   219      0        4           (”, NN)   NN         ”
        5095     0        11          (”, NN)   NN         ”
        5199     0        4           (”, NN)   NN         ”
        3651     0        7           (”, NN)   NN         ”
        3704     0        5           (”, NN)   NN         ”

[3513063 rows x 3 columns]

In [ ]:
TOKEN[TOKEN.token_str == '″']

,,,,pos_tuple,pos,token_str
book_id,para_num,sent_num,token_num,,,


In [377]:
# patternDel = "/[!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~]/g"
# filter = temp['token_str'].str.match(r'<quote>') #https://www.codegrepper.com/code-examples/javascript/regex+find+all+punctuation
# filter

# patternDel = "(^*[1-9!'"'"'"])"
# filter = temp['token_str'].str.contains(patternDel)
# filter

In [378]:
# remove punctuation and integers
# https://stackoverflow.com/questions/39948757/how-to-delete-rows-in-python-pandas-dataframe-using-regular-expressions

# patternDel = "(^*[1-9!'"'"'"])"
# filter = df['Event Name'].str.contains(patternDel)

# df = df[~filter]

In [ ]:
VOCAB = TOKEN['token_str'].value_counts()\
    .to_frame()\
    .reset_index()\
    .rename(columns={'token_str':'n', 'index':'term_str'})\
    .sort_values('term_str')
    
VOCAB.index.name = 'term_id'

In [ ]:
n_tokens = VOCAB.n.sum()
VOCAB['p'] = VOCAB['n'] / n_tokens
VOCAB['log_p'] = np.log2(VOCAB['p'])

In [ ]:
VOCAB.head()

,term_str,n,p,log_p
term_id,,,,
100,A,4277,1.217456e-03,-9.681915
47080,AB,1,2.846519e-07,-21.744298
39729,ABANDON,1,2.846519e-07,-21.744298
33967,ABBEY,2,5.693038e-07,-20.744298
36300,ABBOT,2,5.693038e-07,-20.744298


In [ ]:
TOKEN.to_csv('./TOKEN.csv')

In [ ]:
LIBRARY.to_csv('./LIBRARY.csv')

In [ ]:
# vocab table?? term_id	term_rank	term_str	n	num	stop	stem_porter	stem_snowball	stem_lancaster	pos_max	term_rank2	p	zipf_k	zipf_k2	zipf_k3	p2	h	df	idf	tfidf_sum	h2	x_factor2	log of term_rank2
VOCAB.to_csv('./VOCAB.csv')

In [ ]:
LIBRARY.shape

NameError: name 'LIBRARY' is not defined